# Laplace

Notebook to perform Laplace transformations

Author: Lucas Schneider

---

## Initialization

In [1]:
import sympy as sym
import numpy as np

from sympy.integrals import laplace_transform
from sympy.integrals import inverse_laplace_transform

from IPython.display import display
from IPython.display import Math
from sympy.interactive import printing

# sym.init_printing()

In [2]:
t_var = sym.symbols('t', real=True)
s_var = sym.symbols('s')

## Direct Transform

In [ ]:
# Other symbols declaration
phi = sym.symbols('φ', real=True)

In [ ]:
# Input
expression = sym.cos(t_var)

expression *= sym.Heaviside(t_var)
print(str(expression).replace('**','^'))
expression

In [ ]:
U = laplace_transform(expression, t_var, s_var)
U[0]

## Inverse Transform

In [3]:
def poly_from_list(num_list, den_list, latex=True, string=False):
    num_pol = sym.Poly(num_list, s_var)
    den_pol = sym.Poly(den_list, s_var)

    F = num_pol / den_pol
    F_str = str(F).replace('**','^').replace('I', 'i').replace('exp', 'e^')
    
    if latex:
        display(Math(f'F(s) = {printing.default_latex(F)}'))

    if string:
        print(f'String for Wolfram: {F_str}\n')

    return num_pol, den_pol

In [4]:
def partial_fractions(num_pol, den_pol, latex=True, string=False):
    sum_of_complete, sum_of_fraction = sym.div(num_pol, den_pol)

    sum_of_fraction = sym.apart(sum_of_fraction / den_pol, full=True).doit()

    fraction_terms = list(sum_of_fraction.args)
    fraction_terms = [frac.simplify() for frac in fraction_terms]

    complete_terms = [list(reversed(sum_of_complete.coeffs()))[i] * s_var ** i for i in range(len(sum_of_complete.coeffs()))]

    F = sum(complete_terms) + sum(fraction_terms)
    F_str = str(F).replace('**','^').replace('I', 'i').replace('exp', 'e^')

    if latex:
        display(Math(f'F(s) = {printing.default_latex(F)}'))

    if string:
        print(f'String for Wolfram: {F_str}\n')

    return complete_terms, fraction_terms


In [5]:
def residue(num_pol, den_pol, latex=True, string=False):
    residues = []
    poles = []
    multi = []

    terms = partial_fractions(num_pol, den_pol, latex, string)

    complete_terms, fraction_terms = terms

    roots_dict = sym.polys.polyroots.roots(den_pol)
    for root in roots_dict.keys():
        for i in range(roots_dict[root]):
            poles.append(root)
            multi.append(i+1)

    for i in range(len(poles)):
        for frac in fraction_terms:
            possible_residue = (frac * ((s_var - poles[i]) ** multi[i])).simplify()
            if str(s_var) not in str(possible_residue):
                # print(possible_residue, poles[i], sep='   ')
                residues.append(possible_residue)

    # TODO: Print residues table (with Pandas?)

    res = residues, poles, multi

    return terms, res

In [6]:
N_list = [20, 60]
D_list = [1, 5, 13, 19, 10]
N_pol, D_pol = poly_from_list(N_list, D_list)
terms, res = residue(N_pol, D_pol)
print(res)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

([10, -4, -3 - I, -3 + I], [-1, -2, -1 - 2*I, -1 + 2*I], [1, 1, 1, 1])


In [ ]:
def conjugate_already_seen(pole, multi, poles_seen):
    for pole_seen, multi_seen in poles_seen:
        if (pole.equals(pole_seen) or (pole.conjugate()).equals(pole_seen)) and (multi == multi_seen):
            return True

    return False

In [ ]:
f_terms = []

complex_poles_done = []
for i in range(len(residues)):
    if not (sym.im(poles[i])).equals(0):
        if not conjugate_already_seen(poles[i], multi[i], complex_poles_done):
            if sym.im(residues[i]) < 0:
                Ak = residues[i].conjugate()
                pk = poles[i].conjugate()
            else:
                Ak = residues[i]
                pk = poles[i]

            # t_var ** (multi[i] - 1) / sym.factorial(multi[i] - 1)
            term = sym.Mul(t_var ** (multi[i] - 1) / sym.factorial(multi[i] - 1), 2 * sym.sqrt(sym.re(Ak) ** 2 + sym.im(Ak) ** 2), sym.exp(sym.re(pk) * t_var), sym.cos(sym.Add(sym.im(pk)*t_var, sym.arg(Ak), evaluate=False)))#, evaluate=False)

            # Para conferir com o Wolfram
            # term_check = sym.Mul(residues[i] / sym.factorial(multi[i] - 1), t_var ** (multi[i] - 1), sym.exp(poles[i] * t_var), evaluate=False)
            # term_check += sym.Mul(residues[i+1] / sym.factorial(multi[i+1] - 1), t_var ** (multi[i+1] - 1), sym.exp(poles[i+1] * t_var), evaluate=False)
            # print(term_check)
            # print(str((term_check - term).simplify()).replace('I','i'))

            print(Ak, pk, multi[i], str(term).replace('exp', 'e^'), sep='\n')
            print('')

            complex_poles_done.append((poles[i], multi[i]))
        else:
            continue
    else:
        term = sym.Mul(residues[i] / sym.factorial(multi[i] - 1), t_var ** (multi[i] - 1), sym.exp(poles[i] * t_var), evaluate=False)
    
    # print(term)
    # if multi[i] == 2:
    f_terms.append(term)

print(str(sum(f_terms)).replace('**','^').replace('exp', 'e^'))
sum(f_terms)

In [ ]:
f_terms = []

for i in range(len(residues)):
    term = sym.Mul(residues[i] / sym.factorial(multi[i] - 1), t_var ** (multi[i] - 1), sym.exp(poles[i] * t_var), evaluate=False)
    # print(term)
    
    # if multi[i] == 2:
    f_terms.append(term)

print(str(sum(f_terms)).replace('**','^').replace('I', 'i').replace('exp', 'e^'))
sum(f_terms)

In [ ]:
# Apply the Inverse Laplace Transform from sympy
f_complete_terms = []
f_fraction_terms = []

for term in complete_terms:
    transform = inverse_laplace_transform(term, s_var, t_var)
    f_complete_terms.append(transform)

for term in fraction_terms:
    transform = inverse_laplace_transform(term, s_var, t_var)
    f_fraction_terms.append(transform)

f = sum(f_complete_terms) + sum(f_fraction_terms)
print(str(f).replace('**','^'))
f

In [ ]:
f.evalf(subs={t_var: 0.5})